In [ ]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier

# Idea
Representar los postulantes y avisos mediantes sus propios atributos mas relaciones entre ellos
- Usuarios
    - Datos propios
        - educacion: Superior bool (universitario, posgrado, master, doctorado)
        - edad
        - genero: MASC o FEM bool
    - Datos de avisos
        - Areas mas vistas dentro de las mas postuladas

- Avisos
    - Datos propios:
        - Areas dentro del top
        - Tipo: FullTime bool
        - Zona: GBA bool
    - Datos de usuarios
        - cantidad de hombres/mujeres
        - edad promedio
        - nivel academico
 

In [ ]:
# Constantes
CANTIDAD_DE_AREAS = 10
FRAC = 0.3
FACTOR_NO_POST = 1

# 1. Cargar datos y filtrar outlayers

## 1.1 Genero y edad

In [ ]:
genyedad = pd.concat([
    pd.read_csv("../input/datos/fiuba_2_postulantes_genero_y_edad.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_2_postulantes_genero_y_edad.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_2_postulantes_genero_y_edad.csv")
])
genyedad.head()

In [ ]:
# Se filtran los datos sin sexo declarado
genyedad = genyedad[(genyedad['sexo'] == 'FEM') | (genyedad['sexo'] == 'MASC')]

In [ ]:
genyedad['sexo'].value_counts()

In [ ]:
# Calculamos las edades
genyedad['fechanacimiento'] = pd.to_datetime(genyedad['fechanacimiento'], errors='coerce')

In [ ]:
genyedad['edad'] = 2018 - genyedad['fechanacimiento'].dt.year
genyedad.head()

In [ ]:
# Nos quedamos solo con las edades entre 18 y 75
#genyedad = genyedad[(genyedad['edad'] >= 18) & (genyedad['edad'] <= 75)]
#genyedad.head()
genyedad.fillna(genyedad['edad'].mean(), inplace=True)

In [ ]:
genyedad.drop(columns='fechanacimiento', inplace=True)

In [ ]:
genyedad.drop_duplicates(subset=['idpostulante'], inplace=True)

In [ ]:
genyedad.head()

In [ ]:
# SEXO
# 1 MASC
# 0 FEM
genyedad['sexo'] = genyedad['sexo'].apply(lambda x: 1 if x == 'MASC' else 0)
genyedad.head()

In [ ]:
genyedad.info(memory_usage='deep')

In [ ]:
# Reducimos el tamanio de edad a 1byte.
genyedad = genyedad.astype({'edad':'int8'})
genyedad.info(memory_usage='deep')

In [ ]:
genyedad.head()

## 1.2 Educacion

In [ ]:
educacion = pd.concat([
    pd.read_csv("../input/datos/fiuba_1_postulantes_educacion.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_1_postulantes_educacion.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_1_postulantes_educacion.csv")
])
educacion.head()

In [ ]:
# Me quedo solo con los graduados
educacion = educacion[educacion['estado'] == 'Graduado']
educacion.head()

In [ ]:
educacion['nombre'].value_counts()

In [ ]:
# edu
# 1 Educacion superior: universitario, master, posgrado, doctorado
# 0 Secundario, terciario/tecnico, otros
superior = ['Universitario', 'Posgrado', 'Master', 'Doctorado']
educacion['edu'] = educacion['nombre'].apply(lambda x: 1 if x in superior else 0)
educacion.head()

In [ ]:
educacion.drop(columns=['nombre', 'estado'], inplace=True)

In [ ]:
educacion.drop_duplicates(subset=['idpostulante'], inplace=True)
educacion.head()

## 1.3 Generacion de datos de usuarios

In [ ]:
usuarios = pd.merge(genyedad, educacion, on='idpostulante', how='outer')
usuarios.head()

In [ ]:
usuarios[['sexo']] = usuarios[['sexo']].fillna(bool(random.getrandbits(1)))
usuarios[['edad']] = usuarios[['edad']].fillna(np.random.normal(usuarios['edad'].mean(), np.sqrt(usuarios['edad'].var())))
usuarios[['edu']] = usuarios[['edu']].fillna(bool(random.getrandbits(1)))
usuarios.info()

Con la informacion presente caracterizamos propiamente a los usuarios de navent.

## 1.5 Avisos

In [ ]:
avisos = pd.concat([
    pd.read_csv("../input/datos/fiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/MISfiuba_6_avisos_detalle_missing_nivel_laboral.csv")
])
avisos.head()

In [ ]:
avisos.drop_duplicates(subset='idaviso', inplace=True)

In [ ]:
# Eliminamos las columnas con gran contenido nulo o con poco aporte
avisos.drop(columns=['idpais', 'titulo', 'descripcion', 
                     'ciudad', 'mapacalle', 'denominacion_empresa' ],
           inplace=True)
avisos.head()

In [ ]:
# Nombre de zona: La gran mayoria se centra en el Gran Buenos Aires
# 1 GBA 
# 0 otro
avisos['nombre_zona'] = avisos['nombre_zona'].apply(lambda x: 1 if x == 'Gran Buenos Aires' else 0)
avisos.head()

In [ ]:
# Tipo de trabajo: La gran mayoria es del tipo Full-time
# 1 Full-time
# 0 otro
avisos['tipo_de_trabajo'] = avisos['tipo_de_trabajo'].apply(lambda x: 1 if x == 'Full-time' else 0)
avisos.head()

In [ ]:
# Nivel laboral: La gran mayoria es del tipo senio
# 1 Senior/ Semi-Senior
# 0 otros
avisos['nivel_laboral'] = avisos['nivel_laboral'].apply(lambda x: 1 if x == 'Senior / Semi-Senior' else 0)
avisos.head()

In [ ]:
# Para seleccionar las areas nos quedaremos con las areas mas postuladas, y por lo tanto
# pasaremos a cargar postulaciones primero

## 1.6 Postulaciones

In [ ]:
postulaciones = pd.concat([
    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
])
postulaciones.head()

In [ ]:
postulaciones.drop(columns='fechapostulacion', inplace=True)
postulaciones.head()

In [ ]:
postulaciones.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)

In [ ]:
mas_postuladas = pd.merge(postulaciones, avisos[['idaviso','nombre_area']], on='idaviso', how='left')
mas_postuladas.head()

In [ ]:
mas_postuladas.info(memory_usage='deep')

In [ ]:
mas_postuladas['nombre_area'].value_counts().head(CANTIDAD_DE_AREAS)

In [ ]:
areas_mas_postuladas = mas_postuladas['nombre_area'].value_counts().head(CANTIDAD_DE_AREAS).index
areas_mas_postuladas

In [ ]:
'Ventas' in areas_mas_postuladas

In [ ]:
# Luego realizamos one-hot-encoding sobre las areas dejando a las demas como otro
avisos['nombre_area'] = avisos['nombre_area'].apply(lambda x: x if x in areas_mas_postuladas else 'Otro')
avisos.head()

In [ ]:
dummies_area = pd.get_dummies(avisos['nombre_area'])
dummies_area.head()

In [ ]:
avisos = pd.concat([avisos, dummies_area], axis=1)
avisos.head()

In [ ]:
avisos.drop(columns='nombre_area', inplace=True)

In [ ]:
# Con estos datos caracterizamos propiamente a los avisos
avisos.head()

# 2. Relaciones cruzadas

1. ## 2.1 Usuarios: Proporcion de vistas a areas, tipo de trabajo y nivel laboral

In [ ]:
vistas = pd.concat([
    pd.read_csv("../input/datos/fiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_3_vistas.csv")
])
vistas.head()

In [ ]:
vistas.drop(columns='timestamp', inplace=True)
vistas.head()

In [ ]:
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas.head()

In [ ]:
avisos_vistos = pd.merge(avisos, vistas, on='idaviso', how='right')
avisos_vistos.head()

In [ ]:
avisos_vistos = avisos_vistos.drop(columns='idaviso').pivot_table(index='idpostulante', aggfunc='mean')

In [ ]:
usuarios = pd.merge(usuarios, avisos_vistos, on='idpostulante', how='left')
usuarios.head()

In [ ]:
usuarios.info()

In [ ]:
usuarios.iloc[:,4:] = usuarios.iloc[:,4:].fillna(0)
usuarios.info()

In [ ]:
#usuarios.iloc[:,1:] = usuarios.iloc[:,1:].astype('float16')

In [ ]:
usuarios.head()

In [ ]:
col_us = usuarios.columns[1:]
nuevos = ['idpostulante']
for c in col_us:
    nc = c + '_us'
    nuevos.append(nc)
print(nuevos)
usuarios.columns = nuevos

In [ ]:
usuarios.head()

Con estos datos caracterizamos a los usuarios tanto con sus propios datos como con las relaciones cruzadas con avisos vistos, que como vimos en el TP1, tienen una estrecha relacion con los avisos a los cuales finalmente se postularan

## 2.2 Avisos: Que tipo de usuarios se postulan

In [ ]:
vistas.head()

In [ ]:
usuarios_visitantes = pd.merge(usuarios[['idpostulante', 'sexo_us', 'edad_us', 'edu_us']], vistas, 
                               on='idpostulante', how='left')
usuarios_visitantes.head()

In [ ]:
usuarios_visitantes = usuarios_visitantes.drop(columns='idpostulante')

In [ ]:
usuarios_visitantes = usuarios_visitantes.astype({'sexo_us':'int8',
                                                 'edu_us':'int8'})

In [ ]:
data_avisos = usuarios_visitantes.pivot_table(index='idaviso', 
                                aggfunc={'sexo_us':'mean', 
                                         'edad_us':'mean',
                                         'edu_us':'mean'})
data_avisos.head()

In [ ]:
avisos.head()

In [ ]:
avisos = pd.merge(avisos, data_avisos, on='idaviso',how='right')
avisos.head()

In [ ]:
#avisos.iloc[:,1:].head()

In [ ]:
#avisos.iloc[:,1:] = avisos.iloc[:,1:].astype('float16')
avisos.head()

In [ ]:
avisos.iloc[1].head()

In [ ]:
#avisos.replace([np.inf, -np.inf], np.nan, inplace=True)
#avisos[['sexo_us']] = avisos[['sexo_us']].fillna(avisos['sexo_us'].mean())
#avisos[['edad_us']] = avisos[['edad_us']].fillna(avisos['edad_us'].mean() )
#avisos[['edu_us']] = avisos[['edu_us']].fillna(avisos['edu_us'].mean())
#avisos['edad_us'] = avisos['edad_us'].apply(lambda x: genyedad['edad_us'].mean() if (x > 75 or x < 18) else x)
#avisos.head()


De esta forma caracterizamos a los avisos segun sus propios atributos mas las caracteristicas de los usuarios que los visitan

# 3. Armado de datos postulados

In [ ]:
usuarios.head()

In [ ]:
avisos.head()

In [ ]:
datos_postulados = pd.merge(postulaciones, usuarios, on='idpostulante', how='right')
datos_postulados.head()

In [ ]:
datos_postulados.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)
datos_postulados.head()

In [ ]:
datos_postulados = pd.merge(datos_postulados, avisos, on='idaviso', how='right')
datos_postulados['postulado'] = 1

In [ ]:
datos_postulados.head()

# 4. Generacion de NO_POSTULADOS

In [ ]:
datos_post = datos_postulados.sample(frac=FRAC)
datos_post.info()

In [ ]:
# renombro para que coincida con los nombres anteriores
columnas_us = usuarios.columns
columnas_us

In [ ]:
nuevos = ['idpostulante']
for c in columnas_us[1:]:
    nuevo_c = c + '_x'
    nuevos.append(nuevo_c)
usuarios.columns=nuevos
usuarios.head()

In [ ]:
avisos_columnas = avisos.columns
nuevos  = ['idaviso']
for c in avisos_columnas[1:]:
    nuevo_c = c + '_y'
    nuevos.append(nuevo_c)
print(nuevos)
avisos.columns=nuevos
avisos.head()

In [ ]:
N_NO_POST = int(datos_post['postulado'].value_counts()[1] * FACTOR_NO_POST)
usuarios_no_post = usuarios.sample(n=N_NO_POST, replace=True)
avisos_no_post = avisos.sample(n=N_NO_POST, replace=True)
usuarios_no_post.reset_index(drop=True, inplace=True)
avisos_no_post.reset_index(drop=True, inplace=True)

In [ ]:
datos_no_post = pd.concat([usuarios_no_post, avisos_no_post], axis=1)
datos_no_post['postulado'] = 0
datos_no_post.head()

# 5. Datos para el algoritmo

In [ ]:
datos_post.reset_index(drop=True, inplace=True)
datos_no_post.reset_index(drop=True, inplace=True)
datos = pd.concat([datos_post, datos_no_post])
datos.head()

In [ ]:
datos['postulado'].value_counts()

# 6. Recopilacion de datos a predecir

In [ ]:
final100k = pd.read_csv("../input/finalfinal/test_final_100k.csv")
final100k.info()

In [ ]:
avisos.head()

In [ ]:
usuarios.head()

In [ ]:
df1 = pd.merge(final100k, usuarios, on='idpostulante', how='left')
df1.info()

In [ ]:
df2 = pd.merge(df1, avisos.drop_duplicates(subset='idaviso'), on='idaviso', how='left')
final100k = df2

In [ ]:
final100k.fillna(0, inplace=True)

In [ ]:
final100k.info()

In [ ]:
# Por alguna razon tengo columnas en ordenes distintos.
# orden final de columnas
datos.sort_index(axis=1,inplace=True)
final100k.sort_index(axis=1, inplace=True)

# 7. Algoritmos de Machine Learning

In [ ]:
X = datos.drop(columns=['idpostulante', 'idaviso', 'postulado'])

In [ ]:
X = X.fillna(0)
X.drop(columns=['nivel_laboral', 'nombre_zona'], inplace=True)

In [ ]:
y = datos['postulado']
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

In [ ]:
#for i in range(-1, 10):
i = 1
alpha = 10**(-i)  
print(alpha)
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)  
print('accuracy: {0:.2f}%'.format(accuracy_score(y_test,y_pred)*100))

In [ ]:
final100k.head()

In [ ]:
# PREDICION
X_final = final100k.drop(columns=['id', 'idpostulante', 'idaviso'])

y_final = clf.predict(X_final)
y_final

# 8. SUMMIT

In [ ]:
y_summit = pd.DataFrame(y_final)
y_summit.columns = ['sepostulo']
summit = y_summit.reset_index()
summit.head()
summit.rename(columns={'index':'id'}, inplace=True)
summit.to_csv("summitGSNB.csv", index=False)


In [ ]:
summit.info()